# References
* This notebook shows how to use the `classifierclass.py` file to trian the spike features extracted from `network.py`.
* [Neural Nets](http://neuralnetworksanddeeplearning.com/chap3.html)
* [Randombackprop](https://github.com/xuexue/randombp/blob/master/randombp.py)
* [Randombackprop](https://github.com/sangyi92/feedback_alignment/blob/master/RFA.ipynb)
* [Backprop](http://blog.aloni.org/posts/backprop-with-tensorflow/)
* [Initializers](https://towardsdatascience.com/hyper-parameters-in-action-part-ii-weight-initializers-35aee1a28404)
* [Dropout](https://github.com/pinae/TensorFlow-MNIST-example/blob/master/fully-connected.py)
* [Softmax](https://stackoverflow.com/questions/34240703/what-is-logits-softmax-and-softmax-cross-entropy-with-logits)
* [SoftmaxLogits](https://www.tensorflow.org/api_docs/python/tf/nn/softmax_cross_entropy_with_logits)

# Imports

In [8]:
import numpy as np
import h5py, sys, os, time, pickle
GPU =      False
sys.path.insert(0, "/home/ruthvik/Desktop/Summer 2017/SpykeFlow")
USE_FASTER_METHOD =   True
if(not GPU):
    os.environ["CUDA_VISIBLE_DEVICES"]="-1"
import numpy as np
import spykeflow as sf
import matplotlib.pyplot as plt
from spykeflow import nn_classifierclass as cls

# Load Data

In [10]:
filename = '../../spiking_networks/emnist_train_pool1_acc_cont_spike_features_inh_False_conv1maps_30.h5'
with h5py.File(filename, 'r') as hf:
    emnist_train_images = hf['pool1_spike_features'][:].astype(np.float32)

#emnist_train_images[np.where(emnist_train_images>=1)] = 1
print('Total train features:{}'.format(emnist_train_images.shape[0]))


filename = '../../spiking_networks/emnist_test_pool1_acc_cont_spike_features_inh_False_conv1maps_30.h5'
with h5py.File(filename, 'r') as hf:
    emnist_test_images = hf['pool1_spike_features'][:].astype(np.float32)
#emnist_test_images[np.where(emnist_test_images>=1)] = 1
print('test features:{}'.format(emnist_test_images.shape[0]))


filehandle = open('../../spiking_networks/emnist_train_y.pkl','rb')
emnist_train_labels = pickle.load(filehandle).astype(np.int).tolist()
filehandle.close()
#emnist_train_labels = to_categorical(emnist_train_labels, num_classes=47)


filehandle = open('../../spiking_networks/emnist_test_y.pkl','rb')
emnist_test_labels = pickle.load(filehandle).astype(np.int)
filehandle.close()
#emnist_test_labels = to_categorical(emnist_test_labels, num_classes=47)
print('Length of test labels:{}'.format(emnist_test_labels.shape[0]))

Total train features:112799
test features:18800
Length of test labels:18800


# Setup a NN classifier and train

In [ ]:
n_classes = 47
n_hidden = 1
network_structure = [train_pool1_spike_features.shape[1],1500,n_classes]
#network_structure = [train_pool1_spike_features.shape[1],n_classes]
#activation_fns = ['sigmoid']*(n_hidden)+['softmax']
#activation_fns = ['tanh']*(n_hidden)+['softmax']
activation_fns = ['relu']*(n_hidden)+['softmax']
#activation_fns = ['swish']*(n_hidden)+['softmax']
#activation_fns = ['softmax']
#sys.exit()
weight_init = 'he_uniform'
eta_drop_type = 'plateau'
lmbda = 0.000
batch_size = 5
eta = 0.001

log_path = '/home/ruthvik/Desktop/Summer 2017/tf_graph_outputs/emnist/emnist_regular_backprop_final'+''.\
join(activation_fns+['-',weight_init,'-',eta_drop_type,str(-lmbda)])+'/eta'+str(-eta)
print(log_path)
neural_net = cls.Classifier(train_data=(emnist_train_images,emnist_train_labels),
                            test_data=(emnist_test_images,emnist_test_labels),
                            network_structure=network_structure,activation_fns=activation_fns,
                            epochs=10,eta=eta,lmbda=lmbda,verbose=1,plots=True,optimizer='adam',
                            eta_decay_factor=1.007,patience=8,eta_drop_type=eta_drop_type,
                            epochs_drop=1, val_frac=0.09,drop_out=0.0,ip_lyr_drop_out=0.0,
                            leaky_alpha=0.1,leaky_relu=False,weight_init=weight_init,
                            bias_init=0.1,batch_size=batch_size,log_path=log_path)
neural_net.keras_fcn_classifier()